# Definining the time periods for the creation of products

They are several ways to define the periods over which the climatologies have to be computed.<br> Common ways are:
* monthly climatology, aggregating all observations per month,
* seasonal climatology,
* yearly climatology,
* decadal climatology.

If the data coverage is sufficient, one can also make a seasonal climatology per decades, which allows to resolve the seasonal cycle and long term changes.

In `DIVAnd`, the temporal aggregation is represented by a structure called time selector. The most common is `TimeSelectorYearListMonthList` which behaves similarly than the `yearlist` and `monthlist` files the [Fortran version of `DIVA`](https://github.com/gher-uliege/DIVA).

In this notebook we present different approaches:
1. `TimeSelectorYW`,
2. `TimeSelectorYearListMonthList` and 
3. `TimeSelectorRunningAverage`.     

The most common is `TimeSelectorYearListMonthList` (which behaves similarly than the `yearlist` and `monthlist` files in the Fortran version of `DIVA`).

In [ ]:
using DIVAnd
using Dates
using Statistics

## Specify lists of months and of years
Use `TimeSelectorYearListMonthList`.      
Let's work on two time periods: 1970-1990 and 1991-2010, on a monthly basis

In [ ]:
yearlist = [[1970:1990], [1991:2010]];
monthlists = 1:12;
TS1 = DIVAnd.TimeSelectorYearListMonthList(yearlist, monthlists)

In [ ]:
@show length(TS1);

Another example:

In [ ]:
yearlist = [1900:2017]
monthlist = [1:3, 4:6, 7:9, 10:12]
TS1b = DIVAnd.TimeSelectorYearListMonthList(yearlist, monthlist);
@show length(TS1b);

Assume that we have a time vector with these dates:

In [ ]:
obstime = [DateTime(2001, 4, 1), DateTime(2002, 2, 1), DateTime(2018, 3, 1)]

Which observation would be used for the first winter analysis?

In [ ]:
sel = DIVAnd.select(TS1b, 1, obstime)

In [ ]:
obstime[sel]

Note that a time instance in the "center" of a given time interval is given by `DIVAnd.ctimes(TS)`.      
These dates are saved in the NetCDF file together with the `climatology_bounds` from the [NetCDF CF convention](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#climatological-statistics).

In [ ]:
DIVAnd.ctimes(TS1b)

In [ ]:
yearlist = [y:y+9 for y = 1950:10:2000]

In [ ]:
yearlist2 = []
for y = 1950:10:2020
    push!(yearlist2, y:y+9)
end
@show yearlist2

Note that the duration of every year range is 10 years because the upper bound is inclusive.      
The last year range covers the 10 years:

In [ ]:
collect(yearlist[end])'

TS = DIVAnd.TimeSelectorYearListMonthList(yearlist, monthlist);

For this time selector, there are now $4 × 6=24$ time slices

In [ ]:
length(TS)

DIVAnd.ctimes(TS)[1:3]

## Specify lists of months and years with a time window
Use `TimeSelectorYW`.     
Let's work with 10-year window periods centered on 1950, 1960, 1970... 

In [ ]:
years = 1950:10:2010;
yearwindow = 10;
monthlists = 1:12;
TS2 = TimeSelectorYW(years, yearwindow, monthlists)

Note that with `TimeSelectorYW`, we can almost obtain the same solution as the 1st case:

In [ ]:
TS2b = TimeSelectorYW([1980, 2000], 20, monthlists)

## Specify the total year range and the total window
The difference with the previous case is that the minimal and maximal years are conserved in the created periods.     
Thanks to Lennert (VLIZ) for providing the example and the code.     

In [ ]:
function yearlists_(dataset_range, total_window_yrs)
    # dataset_range = 2000:2012
    # total_window_yrs = 10
    # will return: [2000:2009, 2001:2010, 2002:2011, 2003:2012]

    n_windows = length(dataset_range) - total_window_yrs + 1
    a = Array{UnitRange{Int64},1}(undef, n_windows)

    for i = 1:n_windows
        a[i] = dataset_range[i]:(dataset_range[i]+total_window_yrs-1)
    end
    return (a)
end

In [ ]:
yearlists = yearlists_(1990:2010, 10);
TS3 = TimeSelectorYearListMonthList(yearlists, monthlists)
@show(TS3.yearlists[1]);
@show(TS3.yearlists[2])

## Time aggregation in climatologies

For a climatology, there are different ways to aggregate data in time. Common ways are:
* monthly climatology, aggregating all observations per month
* seasonal climatology
* yearly climatology
* decadal climatology

If the data coverage is sufficient, one can also make a seasonal climatology per decades which allows one to resolve the seasonal cycle and long term changes.

### Overlapping years

Sometimes is it desirable to have overlapping year range to make a climatology similar to a running average.   
This can be achieved by a suitable definition of `yearlist`:

In [ ]:
yearlist = [y:y+5 for y = 1990:2000]

Every time slice is a 6-year average form data from the same season and there are $4 × 11=44$ time slices in this example. 

In [ ]:
monthlist

In [ ]:
TS = DIVAnd.TimeSelectorYearListMonthList(yearlist, monthlist);
length(TS)

Since the data is overlapping, the same observation is used in multiple time instances:

In [ ]:
obstime = [DateTime(2000, 1, 1)]
for n = 1:length(TS)
    nobs = sum(DIVAnd.select(TS, n, obstime))
    if nobs > 0
        println("$nobs observation(s) are used in time slice $n")
    end
end

As expected an observation is used 6 times.